In [1]:
import tensorflow as tf
from tensorflow.keras import Model
from tensorflow.keras.layers import Conv2D, Dense, Flatten
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Mean, SparseCategoricalAccuracy
from tensorflow.keras.utils import plot_model

from tqdm import tqdm

BATCH_SIZE = 32
EPOCHS = 6

2022-11-26 18:18:36.074505: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-26 18:18:36.329623: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-26 18:18:36.340380: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-26 18:18:36.340414: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

### Load data

In [2]:
mnist = tf.keras.datasets.mnist

(train_images, train_labels) , (test_images, test_labels) = mnist.load_data()

train_images = train_images/255.0
train_images = train_images[..., tf.newaxis].astype("float32")

test_images = test_images/255.0
test_images = test_images[..., tf.newaxis].astype("float32")

In [3]:
train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_ds = train_ds.shuffle(10000).batch(BATCH_SIZE)

test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_ds = test_ds.batch(BATCH_SIZE)

2022-11-26 18:18:39.883160: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-11-26 18:18:39.883595: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-26 18:18:39.883675: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2022-11-26 18:18:39.883740: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2022-11-26 18:18:39.883800: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Co

### Build model

In [4]:
class ImageClassfier(Model):
    
    def __init__(self,name):
        
        conv_units = 32
        conv_kernal = 3
        conv_activation = tf.nn.relu

        dense1_units = 128
        dense1_activation = tf.nn.relu

        dense2_units = 10 # last layer, 10 units for 10 classes
        
        super(ImageClassfier, self).__init__(name=name)
        self.conv2d = Conv2D(filters=conv_units, kernel_size=conv_kernal, activation=conv_activation)
        self.flatten = Flatten()
        self.dense1 = Dense(units=dense1_units, activation=dense1_activation)
        self.dense2 = Dense(units=dense2_units)
        
    def call(self, images):
        
        conv_images = self.conv2d(images)
        flatten_images = self.flatten(conv_images)
        dense1_images = self.dense1(flatten_images)
        return self.dense2(dense1_images)
    
model = ImageClassfier(name='image_claffier')

In [5]:
# one_bacth = train_ds.take(1)
# for img, label in one_bacth:
#     pred = model(img)
#     pred = tf.nn.softmax(pred)
#     break

### Optimizer, Loss, Metrics

In [6]:
loss_obj =  SparseCategoricalCrossentropy(from_logits=True)
optimizer = Adam()


In [7]:
train_loss = Mean(name='train_loss')
test_loss = Mean(name='test_loss')

train_acc = SparseCategoricalAccuracy(name="train_acc")
test_acc = SparseCategoricalAccuracy(name="test_acc")

In [8]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        pred = model(images, training=True)
        loss = loss_obj(labels, pred)
        
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))
    
    train_loss(loss)
    train_acc(labels, pred)

In [9]:
@tf.function
def test_step(images, labels):
#     with tf.GradientTape() as tape:
    pred = model(images, training=False)
    loss_t = loss_obj(labels, pred)
        
#     gradients = tape.gradient(loss, model.trainable_variables)
#     optimizer.apply_gradients(gradients, model.trainable_variables)
    
    test_loss(loss_t)
    test_acc(labels, pred)

### Train model

In [10]:
for epoch in tqdm(range(1, EPOCHS+1)):
    train_loss.reset_states()
    test_loss.reset_states()    
    train_acc.reset_states()
    test_acc.reset_states()
    
    for images, labels in train_ds:
        train_step(images, labels)
    
    for t_images, t_labels in test_ds:
        test_step(t_images, t_labels)
    
    print(
        f'Epoch {epoch + 1}, '
        f'Loss: {train_loss.result()}, '
        f'Accuracy: {train_acc.result() * 100}, '
        f'Test Loss: {test_loss.result()}, '
        f'Test Accuracy: {test_acc.result() * 100}'
      )

 17%|███████▌                                     | 1/6 [00:30<02:30, 30.03s/it]

Epoch 2, Loss: 0.13489685952663422, Accuracy: 95.96499633789062, Test Loss: 0.08001945912837982, Test Accuracy: 97.43000030517578


 33%|███████████████                              | 2/6 [00:51<01:38, 24.74s/it]

Epoch 3, Loss: 0.04448465630412102, Accuracy: 98.65499877929688, Test Loss: 0.05787041410803795, Test Accuracy: 98.07999420166016


 50%|██████████████████████▌                      | 3/6 [01:12<01:09, 23.05s/it]

...

Epoch 5, Loss: 0.016739148646593094, Accuracy: 99.4183349609375, Test Loss: 0.05776520445942879, Test Accuracy: 98.29000091552734


 83%|█████████████████████████████████████▌       | 5/6 [01:54<00:21, 21.87s/it]

Epoch 6, Loss: 0.009011219255626202, Accuracy: 99.711669921875, Test Loss: 0.0632745698094368, Test Accuracy: 98.3499984741211


100%|█████████████████████████████████████████████| 6/6 [02:15<00:00, 22.60s/it]

Epoch 7, Loss: 0.00832319725304842, Accuracy: 99.7266616821289, Test Loss: 0.061394188553094864, Test Accuracy: 98.29000091552734


In [11]:
model.summary()

Model: "image_claffier"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             multiple                  320       
                                                                 
 flatten (Flatten)           multiple                  0         
                                                                 
 dense (Dense)               multiple                  2769024   
                                                                 
 dense_1 (Dense)             multiple                  1290      
                                                                 
Total params: 2,770,634
Trainable params: 2,770,634
Non-trainable params: 0
_________________________________________________________________


In [12]:
# test
import numpy as np
for images, labels in test_ds.take(1):
    pred = model.predict(images)
    for pred, actuals in zip(pred, labels):
        print(np.array(pred).argmax(), actuals)

1/1 [==============================] - 0s 54ms/step
7 tf.Tensor(7, shape=(), dtype=uint8)
2 tf.Tensor(2, shape=(), dtype=uint8)
1 tf.Tensor(1, shape=(), dtype=uint8)
0 tf.Tensor(0, shape=(), dtype=uint8)
4 tf.Tensor(4, shape=(), dtype=uint8)
1 tf.Tensor(1, shape=(), dtype=uint8)
4 tf.Tensor(4, shape=(), dtype=uint8)
9 tf.Tensor(9, shape=(), dtype=uint8)
5 tf.Tensor(5, shape=(), dtype=uint8)
9 tf.Tensor(9, shape=(), dtype=uint8)
0 tf.Tensor(0, shape=(), dtype=uint8)
6 tf.Tensor(6, shape=(), dtype=uint8)
9 tf.Tensor(9, shape=(), dtype=uint8)
0 tf.Tensor(0, shape=(), dtype=uint8)
1 tf.Tensor(1, shape=(), dtype=uint8)
5 tf.Tensor(5, shape=(), dtype=uint8)
9 tf.Tensor(9, shape=(), dtype=uint8)
7 tf.Tensor(7, shape=(), dtype=uint8)
5 tf.Tensor(3, shape=(), dtype=uint8)
4 tf.Tensor(4, shape=(), dtype=uint8)
9 tf.Tensor(9, shape=(), dtype=uint8)
6 tf.Tensor(6, shape=(), dtype=uint8)
6 tf.Tensor(6, shape=(), dtype=uint8)
5 tf.Tensor(5, shape=(), dtype=uint8)
4 tf.Tensor(4, shape=(), dtype=uint8